In [43]:
using ITensors, ITensorMPS
using LinearAlgebra
using Random
using DataFrames,CSV
using DelimitedFiles
using Dates

t1 = now()
had = (1/sqrt(2))*[1 1; 1 -1] 
HD = (1/sqrt(2))*[1 1; 1 -1]
IDM = [1 0; 0 1]
HD = kron(HD,IDM)
CNOT = [1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0]
EG = CNOT*HD
wid = 1000
center = 1000
CZ = [1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 -1]

I1 = [1 0 + 0im; 0 1]
s_x = [0 1 + 0im; 1 0]
s_y = [0 -im; im 0]
s_z = [1 + 0im 0; 0 -1]

paulis = [I1,s_x,s_y,s_z]

rx_2 = (sqrt(2)/2)*(IDM-1im.*s_x)
rx_m2 = (sqrt(2)/2)*(IDM+1im.*s_x)
ry_2 = (sqrt(2)/2)*(IDM-1im.*s_y)
ry_m2 = (sqrt(2)/2)*(IDM+1im.*s_y)

s1x = (1/sqrt(2))*[0 1 0; 1 0 1; 0 1 0]
s1y = (1/(sqrt(2)*im))*[0 1 0; -1 0 1; 0 -1 0]
s1z = [1 0 0; 0 0 0; 0 0 -1]
Id = [1 0 0; 0 1 0; 0 0 1]

tau = 50
s1dots2 = kron(s1x,Id)*kron(Id,s1x) + kron(s1y,Id)*kron(Id,s1y) + kron(s1z,Id)*kron(Id,s1z)
Paklt2 = (1/2)*s1dots2 + (1/6)*(s1dots2)^2 + (1/3)*kron(Id,Id)
fpaklt = exp(-Paklt2*tau)

function map_site(i)
    if i == 0 
        return("Up")
    elseif i == 1
        return("Z0")
    elseif i == 2
        return("Dn")
    end
end

function trinaryvec(n,cut)
    ket0 = string(n,base=3,pad=cut)
    kettempc = zeros(Int64,cut)
    for k in range(1,length(ket0))
        kettempc[k] = parse(Int64,ket0[k:k])
    end
    return(kettempc)
end

function form_sop(i,j)
    if i == 1
        temp = s1z
    else
        temp = Id2
    end
    
    for q=i+1:j-1
        temp = kron(temp,exp(im*pi*s1z))
    end
    
    temp = kron(temp,s1z)
    
    for q=j+1:L
        temp = kron(temp,Id)
    end
    return(temp)
end

function ket_sop(ketty,i,j)
    qetty = deepcopy(ketty)
    indices = [k for k=i+1:j-1]
    phase = 1
    for q in indices
        ketty = exp_SZ_proj(ketty,q)
    end
    ketty = SZ_proj(ketty,i)
    ketty = SZ_proj(ketty,j)
    return(inner(qetty,ketty))
end

function ket_nnn_sop(ketty,i,j)
    qetty = deepcopy(ketty)
    indices = [k for k=i+2:j-2]
    phase = 1
    for q in indices
        ketty = exp_SZ_proj(ketty,q)
    end
    ketty = SZ_proj(ketty,i)
    ketty = SZ_proj(ketty,i+1)
    ketty = SZ_proj(ketty,j)
    ketty = SZ_proj(ketty,j+1)
    return(inner(ketty,qetty))
end

function convert_MPS_to_statevector(state)
    statevector = zeros(ComplexF64,3^L)
    for q=1:length(statevector)
        statevector[q] = inner(state_store[q],state)
    end
    return(statevector)
end

function form_sop(i,j)
    if i == 1
    temp = s1z
    else
        temp = Id2
    end
    
    for q=i+1:j-1
        temp = kron(temp,exp(im*pi*s1z))
    end
    
    temp = kron(temp,s1z)
    
    for q=j+1:L
        temp = kron(temp,Id)
    end
    return(temp)
end

function AKLT_force(state,lspin,rspin)
    nl,nr = minimum([lspin,rspin]),maximum([lspin,rspin])
    dimen = length(state)
    state .= state/norm(state)
    A = zeros(ComplexF64,9,3^(nspin-2))
    f = zeros(Int64,9,3^(nspin-2))
    for j in 1:dimen
        l = tob10(splicee(ket[j],nl,nr))
        m = tob10(splicer(ket[j],nl,nr))
        A[l+1,m+1] = state[j] 
        f[l+1,m+1] = j 
    end
    A = fpaklt*A
    for i = 1:size(A)[1]
        for j = 1:size(A)[2]
            state[f[i,j]] = A[i,j]
        end
    end
    return(state/norm(state))
end

function circ_no(i)
    if i <= L
        return(i)
    elseif i == L + 1
        return(1)
    elseif i == L + 2
        return(2)
    end
end

function ITensors.op(::OpName"SZ1", ::SiteType"S=1", s1::Index)
        mat = [1 0 0; 0 0 0; 0 0 -1]
        return itensor(mat,s1',s1)
end

function ITensors.op(::OpName"fpaklt", ::SiteType"S=1", s1::Index, s2::Index)
        mat = fpaklt
        return itensor(mat,s2',s1',s2,s1)
end

function ITensors.op(::OpName"eipSZ1", ::SiteType"S=1", s1::Index)
        mat = exp(im*pi*[1 0 0; 0 0 0; 0 0 -1])
        return itensor(mat,s1',s1)
end

function double_ket_sop_SV(ketty,i,j)
    kitty = (ketty .- 1)
    indices = [k for k=i+2:j-2]
    phase = 1
    for q = 1:length(indices)
        phase = phase*exp(-im*pi*kitty[indices[q]])
    end
    phase = phase*kitty[i]
    phase = phase*kitty[i+1]
    phase = phase*kitty[j]
    phase = phase*kitty[j-1]
    return(phase)
end

function reyniv(state,nth::Vector{Float64},cut)
    # Generate the basis kets, and the Matrix A for SVD
    diml = cut
    dimr = nspin-cut
    A = zeros(ComplexF64,3^dimr,3^diml)
    for j in range(0,dim-1)
#        println(j)
        l = tob10(splitright(ket[j+1],cut))
        m = tob10(splitleft(ket[j+1],cut))
#        println([[l,m],state[j+1]])
        A[l+1,m+1] = state[j+1]
    end

    # Create a temp vector to store the Reyni Entropies
    temp = zeros(Float64,length(nth))

    # Compute the Von-Neumann Entropy and the nth Reyni Entropy
#    println(A)
    evls = svdvals(A)
    for i in range(1,length(nth))
        if nth[i]==1
            temp[i] = -sum(((norm(evls[i]))^2)*log2(norm(evls[i])^2) for i=1:length(evls) if 0 < (norm(evls[i]))^2)
        elseif nth[i]!=Inf
            srn = 0
            for k in 1:length(evls)
                if 0 < ((norm(evls[k]))^(2*nth[i])) srn += ((norm(evls[k]))^(2*nth[i])) end
            end
            temp[i] = (1/(1-nth[i]))*log2(srn)
        elseif nth[i]==Inf
            temp[i] = -log2(maximum(evls))
        end
    end
    return(temp)
end

function MPS_entanglement(psi,b)
    psi = orthogonalize(psi,b)
    if b==1
      U,S,V = svd(psi[b], (siteind(psi,b),))
    else
      U,S,V = svd(psi[b], (linkind(psi, b-1), siteind(psi,b)))
    end
    SvN = 0.0
    for n=1:size(S)[1]
        p = S[n,n]^2
        SvN -= p * log2(p)
    end
    return(SvN)
end

function MPS_negativity_and_entanglement(psi,b)
    psi = orthogonalize(psi,b)
    if b==1
      U,S,V = svd(psi[b], (siteind(psi,b),))
    else
      U,S,V = svd(psi[b], (linkind(psi, b-1), siteind(psi,b)))
    end
    SvN = 0.0    
    ent_negt = 0.0
    for n=1:size(S)[1]
        p = S[n,n]^2
        SvN -= p * log2(p)
        for j=n+1:size(S)[1]
            ent_negt = ent_negt + abs(S[n,n]*S[j,j])
        end
    end
    return([SvN,ent_negt])
end

function ket_nnn_sop(ketty,i,j)
    qetty = deepcopy(ketty)
    indices = [k for k=i+2:j-2]
    phase = 1
    for q in indices
        ketty = exp_SZ_proj(ketty,q)
    end
    ketty = SZ_proj(ketty,i)
    ketty = SZ_proj(ketty,i+1)
    ketty = SZ_proj(ketty,j-1)
    ketty = SZ_proj(ketty,j)
    return(inner(ketty,qetty))
end
        
function trinaryvec(n)
    ket0 = string(n,base=3,pad=nspin)
    for k in range(1,length(ket0))
        kettemp[k] = parse(Int64,ket0[k:k])
    end
    return(kettemp)
end

# Turns the binary list to the base ten number number
function tob10(vecs)
    join(vecs)
    parse(Int, join(vecs); base = 3)
end

# Splits the Vector
function splitleft(state, dimstate)
    statenew = zeros(dimstate)
    for i in 1:dimstate
        statenew[i] = Int64(state[i])
    end
    return round.(Int,statenew)
end

function splitright(state,dimstate)
    statenew = zeros(length(state)-dimstate)
    for i in dimstate+1:length(state)
        statenew[i-dimstate] = Int64(state[i])
    end
    return round.(Int,statenew)
end
# Divides the Vector into Two Parts at left and right
function splicee(state,left,right)
    statevo = zeros(2)
    statevo[1] = state[left]
    statevo[2] = state[right]
    return round.(Int,statevo)
end

function splicer(state,left,right)
    statevo = zeros(length(state)-2)
    jp = 0 
    for i=1:left-1
        jp = jp + 1
        statevo[jp] = state[i]
    end
    for i=left+1:right-1
        jp = jp + 1
        statevo[jp] = state[i]
    end
    for i=right+1:length(state)
        jp = jp + 1
        statevo[jp] = state[i]
    end
    return round.(Int,statevo)
end

function ITensors.op(::OpName"CX", ::SiteType"S=1/2", s1::Index, s2::Index)
  mat = [1 0 0 0
         0 1 0 0
         0 0 0 1
         0 0 1 0]
  return itensor(mat, s2', s1', s2, s1)
end

function ITensors.op(::OpName"SZ1", ::SiteType"S=1", s1::Index)
        mat = [1 0 0; 0 0 0; 0 0 -1]
        return itensor(mat,s1',s1)
end

function ITensors.op(::OpName"fpaklt", ::SiteType"S=1", s1::Index, s2::Index)
        mat = fpaklt
        return itensor(mat,s2',s1',s2,s1)
end

function ITensors.op(::OpName"eipSZ1", ::SiteType"S=1", s1::Index)
        mat = exp(im*pi*[1 0 0; 0 0 0; 0 0 -1])
        return itensor(mat,s1',s1)
end

function exp_SZ_proj(psi,i)
    os = [("eipSZ1",i)]
    psi = ITensors.apply(ITensors.ops(os,sites),psi; cutoff=1e-10)
    return(psi)
end

function SZ_proj(psi,i)
    os = [("SZ1",i)]
    psi = ITensors.apply(ITensors.ops(os,sites),psi; cutoff=1e-10)
    psi = psi
    return(psi)
end

function AKLT_proj(psi,i,j)
    os = [("fpaklt", i, j)]
    psi = ITensors.apply(ITensors.ops(os,sites), psi; cutoff=1e-10)
    psi = psi / sqrt(inner(psi,psi))
    return(psi)
end

function mean(data)
    return(sum(data)/length(data))
end

function ITensors.op(::OpName"proj_plus_x", ::SiteType"S=1", s1::Index)
    mat = [1 0 0; 1 0 0; 1 0 0]
    return itensor(mat,s1',s1)
end

function proj_plus_x(psi,i)
    os = [("proj_plus_x", i)]
    psi = ITensors.apply(ITensors.ops(os,sites), psi; cutoff=1e-10)
    psi = psi / sqrt(inner(psi,psi))
    return(psi)
end

ITensors.state(::StateName"+Z1",::SiteType"S=1") = [1,1,1]

function runy_AKLT(el,pnn,iterations,filename)
    L = el
    global L
    sites = siteinds("S=1",L)
    timesteps = 2*L
    global timesteps
    nspins = L
    nspin = nspins 
    dim = 3^nspin
    global nspin
    global dim
    dimen = 3^L
    zer = [0 1 0]
    mins = [1 0 0] 
    plu = [0 0 1]
    tau = 50
    global tau
    fpaklt = exp(-Paklt2*tau)
    global fpaklt
    global tau
    OP_nn = zeros(Float64,iterations*Int64(L/2))
    OP_nnn = zeros(Float64,iterations*Int64(L/2))    
    Btf = zeros(Float64,iterations)
    negativity = zeros(Float64,iterations)
    bond_dimen_time = zeros(Float64,timesteps)
    bond_dimen_tf = zeros(Float64,iterations)
    for q=1:iterations
        sites = siteinds("S=1",L)
        global sites
        psi = MPS(ComplexF64,sites,["+Z1" for n in 1:L])
#        psi = random_mps(sites; linkdims=3)
        psi = psi/sqrt(inner(psi,psi))
        global psi
        prob_seed = MersenneTwister(abs(rand(Int)))
        for i=1:timesteps
            for q=1:L
#                println([i,q])
                t2 = now()
                if parse(Int64,string(t2 - t1)[1:end-13])  > 7e+7
                    writedlm(string(filename,"Reyni1flop"),"exit!", ", ")
                    return("exit!")
                end
#                println(["que",q])
                p = rand(prob_seed)
                if p < pnn
                    psi = AKLT_proj(psi,circ_no(q),circ_no(q+1))
                else
#                    println([circ_no(q),circ_no(q+2)])
                    psi = AKLT_proj(psi,circ_no(q),circ_no(q+2))
                end
            end
            bond_dimen_time[i] += mean(linkdims(psi))/iterations
        end

        for k=1:Int64(L/2)
            OP_nn[Int64((L/2)*(q-1)+k)] = real(ket_sop(psi,k,Int64(L/2+k-1)))
            OP_nnn[Int64((L/2)*(q-1)+k)] = real(ket_nnn_sop(psi,k,Int64(L/2+k-1)))
        end
        bond_dimen_tf[q] = mean(linkdims(psi))
        temp = MPS_negativity_and_entanglement(psi,Int64(L/2))
    	Btf[q] = temp[1]
        negativity[q] = temp[2]
    end
#    println(OP_nn)
#    println(OP_nnn)
#    println(Btf)
     writedlm(string(filename,"tau=",tau,"pn=",pnn,"it=",iterations,"it_OP_nn_sa_PBC_L2_px_state"),OP_nn, ", ")
     writedlm(string(filename,"tau=",tau,"pn=",pnn,"it=",iterations,"it_OP_nnn_sa_PBC_L2_px_state"),OP_nnn, ", ")
     writedlm(string(filename,"tau=",tau,"pn=",pnn,"it=",iterations,"it_Btf_sa_PBC_L2_px_state"),Btf, ", ")
     writedlm(string(filename,"tau=",tau,"pn=",pnn,"it=",iterations,"it_bond_sa_PBC_L2_px_state"),bond_dimen_tf, ", ")
     writedlm(string(filename,"tau=",tau,"pn=",pnn,"it=",iterations,"bond_time_sa_PBC_L2_px_state"),bond_dimen_time, ", ")
     writedlm(string(filename,"tau=",tau,"pn=",pnn,"it=",iterations,"neg_PBC_L2_px_state"),negativity, ", ")
end
runy_AKLT(initialization,"tada")
t2 = now()
println(t2-t1)

43 milliseconds


In [44]:
t1 = now()
runy_AKLT(4,0.0,1,"butter")
t2 = now()
println(t2-t1)

318 milliseconds


In [41]:
real(ket_sop(psi,1,Int64(L/2)))

0.07388678618321351

In [158]:
linkdims(psi)

5-element Vector{Int64}:
 3
 9
 9
 9
 3

In [34]:
MPS_negativity_and_entanglement(psi,Int64(L/2))

2-element Vector{Float64}:
 0.8544892477411519
 0.5718480589656847

In [13]:
t1 = now()
runy_AKLT(4,0.0,1,"butter")
t2 = now()
println(t2-t1)

LoadError: ArgumentError: input string is empty or only contains whitespace

In [59]:
ITensors.state(::StateName"+Z1",::SiteType"S=1") = [1,1,1]

In [23]:
 psi = random_mps(sites; linkdims=3)

MPS
[1] ((dim=3|id=26|"S=1,Site,n=1"), (dim=3|id=448|"Link,l=1"))
[2] ((dim=3|id=448|"Link,l=1"), (dim=3|id=147|"S=1,Site,n=2"), (dim=3|id=431|"Link,l=2"))
[3] ((dim=3|id=431|"Link,l=2"), (dim=3|id=178|"S=1,Site,n=3"), (dim=3|id=339|"Link,l=3"))
[4] ((dim=3|id=339|"Link,l=3"), (dim=3|id=537|"S=1,Site,n=4"))


In [24]:
array(prod((psi)))

3×3×3×3 Array{Float64, 4}:
[:, :, 1, 1] =
 -0.0700055  -0.0195145  -0.0194954
  0.0410016   0.0191685   0.0310368
  0.0081388   0.0253855   0.0607546

[:, :, 2, 1] =
  0.0614849   0.127398    0.0244144
 -0.0465984  -0.087639   -0.042195
 -0.0372748  -0.0559351  -0.0879821

[:, :, 3, 1] =
 -0.0889822  -0.232833  -0.039382
  0.0722654   0.157978   0.0684707
  0.0676846   0.095857   0.143754

[:, :, 1, 2] =
  0.0168463   0.0505203   0.0948016
 -0.0322187  -0.0366991  -0.100424
 -0.0659306  -0.032637   -0.153608

[:, :, 2, 2] =
 -0.048967    0.114273    0.0154382
  0.0121464  -0.0693687  -0.0161553
 -0.0414405  -0.0242426  -0.0271693

[:, :, 3, 2] =
  0.0233476   0.41142    0.0319917
 -0.052289   -0.266371  -0.0639171
 -0.112589   -0.131817  -0.145564

[:, :, 1, 3] =
 -0.108451    0.317859   -0.0512549
  0.0391808  -0.192487    0.0448345
 -0.0564612  -0.0605232   0.0478795

[:, :, 2, 3] =
  0.0603572  -0.216541    0.0180683
 -0.0163839   0.132821   -0.0117768
  0.0468835   0.0469065  -0.00

In [40]:
t1 = now()
runy_AKLT(10,0.0,1,"butter")
t2 = now()
println(t2-t1)

[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]


In [84]:
L = 8
[mod(div(n-1,2), 2) == 0 ? "Up" : "Dn" for n in 1:L]

8-element Vector{String}:
 "Up"
 "Up"
 "Dn"
 "Dn"
 "Up"
 "Up"
 "Dn"
 "Dn"

In [105]:
t1 = now()
runy_AKLT(10,0.0,1,"butter")
t2 = now()
println(t2-t1)

[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]
[10, 2]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[7, 9]
[8, 10]
[9, 1]


In [36]:
t1 = now()
runy_AKLT(10,1.0,1,"butter")
t2 = now()
println(t2-t1)

[-0.4444896023846339, -0.4444896023848581, -0.4444896023847807, -0.4444896023847168, -0.4444896023846623]
[0.024385287543746387, 0.024385287543792614, 0.02438528754370768, 0.024385287543637613, 0.024385287543579465]
[1.9998538199153815]
1979 milliseconds


In [10]:
t1 = now()
runy_AKLT(10,0.5,1,"butter")
t2 = now()
println(t2-t1)

99139 milliseconds


In [4]:
t1 = now()
runy_AKLT(10,0.5,1,"butter")
t2 = now()
println(t2-t1)

[1, 1]
[1, 2]
[1, 3]
[1, 4]
[1, 5]
[1, 6]
[1, 7]
[1, 8]
[1, 9]
[1, 10]
[2, 1]
[2, 2]
[2, 3]
[2, 4]
[2, 5]
[2, 6]
[2, 7]
[2, 8]
[2, 9]
[2, 10]
[3, 1]
[3, 2]
[3, 3]
[3, 4]
[3, 5]
[3, 6]
[3, 7]
[3, 8]
[3, 9]
[3, 10]
[4, 1]
[4, 2]
[4, 3]
[4, 4]
[4, 5]
[4, 6]
[4, 7]
[4, 8]
[4, 9]
[4, 10]
[5, 1]
[5, 2]
[5, 3]
[5, 4]
[5, 5]
[5, 6]
[5, 7]
[5, 8]
[5, 9]
[5, 10]
[6, 1]
[6, 2]
[6, 3]
[6, 4]
[6, 5]
[6, 6]
[6, 7]
[6, 8]
[6, 9]
[6, 10]
[7, 1]
[7, 2]
[7, 3]
[7, 4]
[7, 5]
[7, 6]
[7, 7]
[7, 8]
[7, 9]
[7, 10]
[8, 1]
[8, 2]
[8, 3]
[8, 4]
[8, 5]
[8, 6]
[8, 7]
[8, 8]
[8, 9]
[8, 10]
[9, 1]
[9, 2]
[9, 3]
[9, 4]
[9, 5]
[9, 6]
[9, 7]
[9, 8]
[9, 9]
[9, 10]
[10, 1]
[10, 2]
[10, 3]
[10, 4]
[10, 5]
[10, 6]
[10, 7]
[10, 8]
[10, 9]
[10, 10]
[11, 1]
[11, 2]
[11, 3]
[11, 4]
[11, 5]
[11, 6]
[11, 7]
[11, 8]
[11, 9]
[11, 10]
[12, 1]
[12, 2]
[12, 3]
[12, 4]
[12, 5]
[12, 6]
[12, 7]
[12, 8]
[12, 9]
[12, 10]
[13, 1]
[13, 2]
[13, 3]
[13, 4]
[13, 5]
[13, 6]
[13, 7]
[13, 8]
[13, 9]
[13, 10]
[14, 1]
[14, 2]
[14, 3]
[14, 4]
[14, 

In [5]:
(95*100)/(60)

158.33333333333334

In [ ]:
t1 = now()
runy_AKLT(12,0.5,1,"butter")
t2 = now()
println(t2-t1)

In [ ]:
t1 = now()
runy_AKLT(14,0.5,1,"butter")
t2 = now()
println(t2-t1)

[1, 1]
[1, 2]
[1, 3]
[1, 4]
[1, 5]
[1, 6]
[1, 7]
[1, 8]
[1, 9]
[1, 10]
[1, 11]
[1, 12]
[1, 13]
[1, 14]
[2, 1]
[2, 2]
[2, 3]
[2, 4]
[2, 5]
[2, 6]
[2, 7]
[2, 8]
[2, 9]
[2, 10]
[2, 11]
[2, 12]
[2, 13]
[2, 14]
[3, 1]
[3, 2]
[3, 3]
[3, 4]
[3, 5]
[3, 6]